# Question 1: Convert XML to a SQL database

Create two tables named `LOW` and `HIGH`, each corresponding to data given for the low and high temperature range.
Each should have the following column names:

- `SPECIES_NAME`
- `TLOW`
- `THIGH`
- `COEFF_1`
- `COEFF_2`
- `COEFF_3`
- `COEFF_4`
- `COEFF_5`
- `COEFF_6`
- `COEFF_7`

Populate the tables using the XML file you created in last assignment. If you did not complete the last assignment, you may also use the `example_thermo.xml` file.

`TLOW` should refer to the temperature at the low range and `THIGH` should refer to the temperature at the high range.  For example, in the `LOW` table, $H$ would have `TLOW` at $300$ and `THIGH` at $1000$ and in the `HIGH` table, $H$ would have `TLOW` at $1000$ and `THIGH` at $5000$.

For both tables, `COEFF_1` through `COEFF_7` should be populated with the corresponding coefficients for the low temperature data and high temperature data.

In [1]:
import xml.etree.ElementTree as ET
import sqlite3
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

In [2]:
tree = ET.parse('../HW9/thermo.xml')
root = tree.getroot()
species = root.find("phase")
species_array = species.find("speciesArray").text
species_name = species_array.split()

species_name

['O', 'O2', 'H', 'H2', 'OH', 'H2O', 'HO2', 'H2O2']

In [3]:
dic = {}
for specie in root.findall("speciesData"):
    for s in specie.findall("species"):
        name = s.get("name")
        dic[name] = {}
        coeffs = s.find("thermo").findall("NASA")
        T_max = []
        T_min = []
        coeff = []
        for c in coeffs:
            T_max.append(c.get("TMAX"))
            T_min.append(c.get("TMIN"))   
            coeff.append(c.find("floatArray").text.split(','))
        dic[name]["TMAX"] = T_max
        dic[name]["TMIN"] = T_min
        dic[name]["coeffs"] = coeff

In [4]:
# dic

In [5]:
db = sqlite3.connect('HW10.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS HIGH")
cursor.execute("DROP TABLE IF EXISTS LOW")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE HIGH (
               SPECIES_NAME TEXT PRIMARY KEY NOT NULL, 
               TLOW TEXT, 
               THIGH TEXT, 
               COEFF_1 TEXT,
               COEFF_2 TEXT,
               COEFF_3 TEXT,
               COEFF_4 TEXT,
               COEFF_5 TEXT,
               COEFF_6 TEXT,
               COEFF_7 TEXT)''')

cursor.execute('''CREATE TABLE LOW (
               SPECIES_NAME TEXT PRIMARY KEY NOT NULL, 
               TLOW TEXT, 
               THIGH TEXT, 
               COEFF_1 TEXT,
               COEFF_2 TEXT,
               COEFF_3 TEXT,
               COEFF_4 TEXT,
               COEFF_5 TEXT,
               COEFF_6 TEXT,
               COEFF_7 TEXT)''')

db.commit() # Commit changes to the database

In [6]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

In [7]:
for key, value in dic.items():
    vals_to_insert_high = (key, float(value["TMIN"][0]), float(value["TMAX"][0]), value["coeffs"][0][0], value["coeffs"][0][1], \
                          value["coeffs"][0][2], value["coeffs"][0][3], value["coeffs"][0][4], \
                          value["coeffs"][0][5], value["coeffs"][0][6])
    cursor.execute('''INSERT INTO HIGH 
                  (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert_high)


In [8]:
HIGH_cols = [col[1] for col in cursor.execute("PRAGMA table_info(HIGH)")]
query_HIGH = '''SELECT * FROM HIGH'''
viz_tables(HIGH_cols, query_HIGH)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,O,1000.0,5000.0,2.56942078E+00,-8.59741137E-05,4.19484589E-08,-1.00177799E-11,1.22833691E-15,2.92175791E+04,4.78433864E+00
1,O2,1000.0,5000.0,3.28253784E+00,1.48308754E-03,-7.57966669E-07,2.09470555E-10,-2.16717794E-14,-1.08845772E+03,5.45323129E+00
2,H,1000.0,5000.0,2.50000001E+00,-2.30842973E-11,1.61561948E-14,-4.73515235E-18,4.98197357E-22,2.54736599E+04,-4.46682914E-01
3,H2,1000.0,5000.0,3.33727920E+00,-4.94024731E-05,4.99456778E-07,-1.79566394E-10,2.00255376E-14,-9.50158922E+02,-3.20502331E+00
4,OH,1000.0,5000.0,3.09288767E+00,5.48429716E-04,1.26505228E-07,-8.79461556E-11,1.17412376E-14,3.85865700E+03,4.47669610E+00
5,H2O,1000.0,5000.0,3.03399249E+00,2.17691804E-03,-1.64072518E-07,-9.70419870E-11,1.68200992E-14,-3.00042971E+04,4.96677010E+00
6,HO2,1000.0,5000.0,4.01721090E+00,2.23982013E-03,-6.33658150E-07,1.14246370E-10,-1.07908535E-14,1.11856713E+02,3.78510215E+00
7,H2O2,1000.0,5000.0,4.16500285E+00,4.90831694E-03,-1.90139225E-06,3.71185986E-10,-2.87908305E-14,-1.78617877E+04,2.91615662E+00


In [9]:
for key, value in dic.items():
    vals_to_insert_low = (key, float(value["TMIN"][1]), float(value["TMAX"][1]), value["coeffs"][1][0], value["coeffs"][1][1], \
                          value["coeffs"][1][2], value["coeffs"][1][3], value["coeffs"][1][4], \
                          value["coeffs"][1][5], value["coeffs"][1][6])
    cursor.execute('''INSERT INTO LOW 
                  (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert_low)

In [10]:
LOW_cols = [col[1] for col in cursor.execute("PRAGMA table_info(LOW)")]
query_LOW = '''SELECT * FROM LOW'''
viz_tables(LOW_cols, query_LOW)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,O,300.0,1000.0,3.16826710E+00,-3.27931884E-03,6.64306396E-06,-6.12806624E-09,2.11265971E-12,2.91222592E+04,2.05193346E+00
1,O2,300.0,1000.0,3.78245636E+00,-2.99673416E-03,9.84730201E-06,-9.68129509E-09,3.24372837E-12,-1.06394356E+03,3.65767573E+00
2,H,300.0,1000.0,2.50000000E+00,7.05332819E-13,-1.99591964E-15,2.30081632E-18,-9.27732332E-22,2.54736599E+04,-4.46682853E-01
3,H2,300.0,1000.0,2.34433112E+00,7.98052075E-03,-1.94781510E-05,2.01572094E-08,-7.37611761E-12,-9.17935173E+02,6.83010238E-01
4,OH,300.0,1000.0,3.99201543E+00,-2.40131752E-03,4.61793841E-06,-3.88113333E-09,1.36411470E-12,3.61508056E+03,-1.03925458E-01
5,H2O,300.0,1000.0,4.19864056E+00,-2.03643410E-03,6.52040211E-06,-5.48797062E-09,1.77197817E-12,-3.02937267E+04,-8.49032208E-01
6,HO2,300.0,1000.0,4.30179801E+00,-4.74912051E-03,2.11582891E-05,-2.42763894E-08,9.29225124E-12,2.94808040E+02,3.71666245E+00
7,H2O2,300.0,1000.0,4.27611269E+00,-5.42822417E-04,1.67335701E-05,-2.15770813E-08,8.62454363E-12,-1.77025821E+04,3.43505074E+00


# Question 2: `WHERE` Statements

1. Write a `Python` function `get_coeffs` that returns an array of 7 coefficients.  
   
   The function should take in two parameters: 1.) `species_name` and 2.) `temp_range`, an indicator variable ('low' or 'high') to indicate whether the coefficients should come from the low or high temperature range.  
   The function should use `SQL` commands and `WHERE` statements on the table you just created in Question 1 (rather than taking data from the XML directly).
```python
def get_coeffs(species_name, temp_range):
    ''' Fill in here'''
    return coeffs
```

2. Write a python function `get_species` that returns all species that have a temperature range above or below a given value. The function should take in two parameters: 1.) `temp` and 2.) `temp_range`, an indicator variable ('low' or 'high').

  When temp_range is 'low', we are looking for species with a temperature range lower than the given temperature, and for a 'high' temp_range, we want species with a temperature range higher than the given temperature.

  This exercise may be useful if different species have different `LOW` and `HIGH` ranges.

  And as before, you should accomplish this through `SQL` queries and where statements.

```python
def get_species(temp, temp_range):
    ''' Fill in here'''
    return coeffs
```

# Question 3: `JOIN` STATEMENTS

Create a table named `ALL_TEMPS` that has the following columns:

- `SPECIES_NAME`
- `TEMP_LOW`
- `TEMP_HIGH`

This table should be created by joining the tables `LOW` and `HIGH` on the value `SPECIES_NAME`.

1. Write a `Python` function `get_range` that returns the range of temperatures for a given species_name.

The range should be computed within the `SQL` query (i.e. you should subtract within the `SELECT` statement in the `SQL` query).
```python
def get_range(species_name):
    '''Fill in here'''
    return range
```

Note that `TEMP_LOW` is the lowest temperature in the `LOW` range and `TEMP_HIGH` is the highest temperature in the `HIGH` range.

In [11]:
db.commit()
# db.close()